Goal: Create a daily NAO index using the CPC methodology 
=====

Author: [Ray Bell](https://github.com/raybellwaves)

Use the time period of the [SubX](http://iridl.ldeo.columbia.edu/SOURCES/.Models/.SubX/) hindcast: 1999-2016.
For a saninty check make sure that 1999-2016 is indeed the hindcast period:

In [ ]:
import xarray as xr
remote_data = xr.open_dataset('http://iridl.ldeo.columbia.edu/SOURCES/.Models'\
                              '/.SubX/.RSMAS/.CCSM4/.hindcast/.zg/dods')
print(remote_data.coords['S'])

The data used here is geopotential height at 500 hPa ($Z_{500}$) from ERA-Interim. This data was simply copied from the University of Reading archive. You can however use the [API](https://software.ecmwf.int/wiki/display/CKB/How+to+download+data+via+the+ECMWF+WebAPI) to download the data from ECMWF.

Two files exists in this directory `ERAI_z500_monthly_1999-2016.nc` (43 Mb) and `b.nc`. `ERAI_z500_monthly_1999-2016.nc` is the monthly mean of $Z_{500}$ January 1996 - December 2016. `b.nc` is daily $Z_{500}$ January 1st 1996 - December 31st 2016. The data has been interpolated to 1° to match the SubX data (360x181) and cut to 20°N-90°N for the analysis. The reference scripts for the data processing beforehand are in the directory `pre_proc`. DO THIS!

CPC's methodology for the daily NAO index can be found [here](http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/history/method.shtml), with some more information [here](http://www.cpc.ncep.noaa.gov/data/teledoc/telepatcalc.shtml). The procedure is based on [Barston and Livezey (1987)](https://journals.ametsoc.org/doi/pdf/10.1175/1520-0493%281987%29115%3C1083%3ACSAPOL%3E2.0.CO%3B2).

My understanding of the methodology is as follows:

1. Use the region of 20°N-90°N with standardized $Z_{500}$ anomalies.

2. Calculate the 10 leading EOFs for each calendar month centred on that month with a window of 3 months. e.g. February is based on January-February-March. Decisions have to be made for January and December: whether to just use the two-month window or leave out the season. (It shouldn't really matter).

3. Rotate the EOFs using the varimax methodology.

4. Linearly interpolate the monthly spatial pattern to the day in question. e.g. February 1$^{st}$ will be linearly interpolated from the DJF pattern (think of the pattern defined mid-way through the mid-month: e.g. January 15$^{th}$) and the JFM pattern (February 15$^{th}$).

5. Least squared regression approach for daily data? (e.g. last paragraph in the NAO/PNA section [here](http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/history/method.shtml) (I don't understand this)

Enter a start-time an end-time for which to calculate the daily NAO index
e.g. lets look at the large negativate NAO of winter 2008/2009:

In [ ]:
stime = '2009-02-01-T12:00:00'
etime = '2009-02-14-T12:00:00'

Now extend for the winter months (DJF)

In [ ]:
stime = '2008-12-01-T12:00:00'
etime = '2009-02-28-T12:00:00'

Now do for all winters